# Import Library

In [88]:
import sqlite3
from time import sleep
import numpy as np
import requests
import json
import pandas as pd
import warnings
import tweepy
import string
import re
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize
warnings.simplefilter(action='ignore', category=FutureWarning)

# Access API

In [2]:

# Authenticate to Twitter
auth = tweepy.OAuthHandler("CkhGa0z1CKMlK7GY56Zxh0250", 
    "Iq1o1IwRzuFw6xCandQbGUX1gzx6GezPD7ek1ptbcw46eni3LB")
auth.set_access_token("492948056-n4H0mlG3efeuJnDKkkoyeW9BWbl7kBrI95Dmlup1", 
    "jMSNiTz1mo764Xnz1x8cEAuu2CQRIhRbstAgRiD5WY7cP")

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


# Important Function

## Koneksi Database

In [10]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Exception as e:
        print(e)
    return conn

In [67]:
## Case Folding

In [94]:
def clean_text(text):
  '''
  DESCRIPTION:
  This function to clean text 
  INPUT: 
  text: string
  OUTPUT: 
  text: string after clean it
  ''' 
  text = text.lower() # convert letters to lower case
  text = re.sub("@[A-Za-z0-9_]+"," ", text) #r emove mention
  text = re.sub("#[A-Za-z0-9_]+"," ", text) # remove hashtag
  text = re.sub(r"\\n"," ",text) # remove \n or enter
  text = re.sub("[^a-zA-Z0]", " ", text) # remove non-letters
  # text = re.sub(r'\d+', '', text) # remove number
  text = re.sub(r'http\S+', '', text) # remove links
  text = re.sub(r"www.\S+", " ", text) # remove link
  text = re.sub("rt"," ",text) # remove RT
  text = text.translate(str.maketrans('','', string.punctuation)) # remove punctuation
  text = re.sub(' +', ' ',text) # remove extra space
  text = text.strip() # remove whitespaces

  # remove stopwords
  stpwds_id = list(set(stopwords.words('indonesian')))
  stpwds_id.append('oh')

  tokens = word_tokenize(text)

  text = ' '.join([word for word in tokens if word not in stpwds_id])

  # Stemming
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  text = stemmer.stem(text)

  return text

## Predict by model

In [11]:
URL = 'http://backend-auto-reply-fp.herokuapp.com/v1/models/auto_reply:predict'
def get_predict(series, URL_Serving):
    X=[]
    for con in series.tolist():
        X.append([con])

    input_data_json = json.dumps({
        "signature_name": "serving_default",
        "instances": X,
        })
        
    r=requests.post(URL_Serving, input_data_json)
    result=r.json()
    result= np.argmax(result['predictions'], axis=1)
    return np.where(result==0,'Neutral', np.where(result==1, 'Negative', 'Positive'))

## Search tweet

In [99]:
def get_tweet(query, api, cursor, result_type='recent', response=True):
    max_id=cursor.execute('SELECT MAX(ID) FROM tweet').fetchall()[0][0]
    data = api.search_tweets(q=query, 
                        result_type=result_type,
                        count=100, 
                        tweet_mode ='extended', 
                        since_id= max_id)
    data= pd.DataFrame([[s.id, s.full_text.replace('\n','').replace('\r',''), s.user.screen_name, s.user.id_str, s.user.followers_count, s.retweet_count, s.favorite_count, s.created_at] for s in data], columns=('ID', 'Texts', 'UserName','UserID', "UserFollowerCount", 'RetweetCount', 'Likes', "CreatedAt"))
    if data.shape[0]>0: 
        data.iloc[:]["clean_text"]=data.iloc[:]['Texts'].apply(clean_text)
        data['sentimen']=get_predict(data['Texts'], URL)
        if response:
            for id in data[(data.sentimen=="Negative")&(data.UserID!='492948056')].ID:
                name=data[data.ID==id]["UserName"].values[0]
                user_id=data[data.ID==id]["UserID"].values[0]
                status="Halo kak @"+name+". maaf atas ketidaknyamanannya. Untuk informasi lebih lanjut, cek dm ya"
                pesan="Maaf kak atas kendala yang dihadapi. Bisa diceritakan lebih detail terkait masalah yang kakak hadapi?"
                try:
                    api.update_status(status, in_reply_to_status_id=id)
                except Exception as error:
                    print(error)
                    print("gagal balas")
                try:
                    api.send_direct_message(user_id,pesan)
                except Exception as error:
                    print(error)
                    print("gagal dm")
    return data[data.UserID!='492948056']
    # else: return "tidak ada tweet"


# Main

## Connect Database

In [13]:
con = create_connection("twit copy.db")
cursor=con.cursor()

## Cek limit request

In [14]:
api.rate_limit_status()['resources']['search']

{'/search/tweets': {'limit': 180, 'remaining': 173, 'reset': 1659312965}}

## Run Constant

In [102]:
while True:
    new_data=get_tweet('"bank t0yip" OR t0yip',api, cursor)
    new_data.to_sql('tweet', con=con, method='multi', if_exists='append', index=False)
    print(f'Data baru sejumlah {new_data.shape[0]}')
    sleep(6)

403 Forbidden
187 - Status is a duplicate.
gagal balas
Data baru sejumlah 3
Data baru sejumlah 0
Data baru sejumlah 0
Data baru sejumlah 0
Data baru sejumlah 0
Data baru sejumlah 0
Data baru sejumlah 0


KeyboardInterrupt: 